### imports e configuração

In [ ]:
import math
import random
import requests
import json
from datetime import datetime, timedelta
from typing import List, Dict, Any, Tuple
import pandas as pd
import numpy as np
from tqdm import tqdm
from shapely.geometry import LineString, Point

# Configurações: endpoints locais
VROOM_BASES = [
    "http://localhost:3000/solve",   # endpoint comum (tentativa)
    "http://localhost:3000/route",
    "http://localhost:3000/solve-route",
    "http://localhost:3000/routes",
    "http://localhost:3000"           # fallback; we'll try POSTs to endpoints in try_vroom_solve
]
OSRM_TABLE_BASE = "http://localhost:5000/table/v1/driving"  # OSRM table service
DEPOT = (-8.738553348981176, -63.88547754489104)  # (lat, lon)

# Seeds
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
def parse_dt(x):
    if pd.isna(x):
        return None
    if isinstance(x, datetime):
        return x
    try:
        return pd.to_datetime(x).to_pydatetime()
    except:
        return None

def secs(td: timedelta):
    return td.total_seconds()

def haversine_m(lat1, lon1, lat2, lon2):
    # meters
    R = 6371000
    phi1 = math.radians(lat1); phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2*R*math.asin(math.sqrt(a))
def try_vroom_solve(payload: dict, timeout=30) -> dict:
    """
    Tenta enviar payload para vários endpoints VROOM locais plausíveis até obter uma resposta válida.
    Retorna o JSON da resposta ou lança erro com último status.
    """
    headers = {"Content-Type": "application/json"}
    last_exc = None
    for url in VROOM_BASES:
        try:
            resp = requests.post(url, json=payload, headers=headers, timeout=timeout)
            if resp.status_code == 200:
                return resp.json()
            # alguns VROOMs respondem 201 or 202; accept 200-299
            if 200 <= resp.status_code < 300:
                return resp.json()
            # otherwise continue trying other endpoints
            last_exc = Exception(f"VROOM {url} status {resp.status_code}: {resp.text[:200]}")
        except Exception as e:
            last_exc = e
    raise last_exc
def osrm_table(coords: List[Tuple[float,float]], annotations: str="duration,distance", sources=None, destinations=None, timeout=30) -> Tuple[np.ndarray, np.ndarray]:
    """
    coords: list of (lat, lon)
    returns (durations_matrix_seconds, distances_matrix_meters) as numpy arrays
    """
    # OSRM expects lon,lat list in path
    coord_str = ";".join([f"{lon},{lat}" for lat,lon in coords])
    params = {"annotations": annotations}
    if sources is not None:
        params["sources"] = ";".join(map(str,sources))
    if destinations is not None:
        params["destinations"] = ";".join(map(str,destinations))
    url = f"{OSRM_TABLE_BASE}/{coord_str}"
    resp = requests.get(url, params=params, timeout=timeout)
    resp.raise_for_status()
    data = resp.json()
    durations = np.array(data.get("durations", []), dtype=float)
    distances = np.array(data.get("distances", []), dtype=float)
    return durations, distances
def build_jobs_from_dfs(df_tecnicos: pd.DataFrame, df_comerciais: pd.DataFrame) -> List[Dict]:
    jobs = []
    idx = 0
    # técnicos
    for _, r in df_tecnicos.iterrows():
        tw_start = parse_dt(r.get("DH_INICIO") or r.get("DH_ALOCACAO") or r.get("DH_CHEGADA"))
        tw_end   = parse_dt(r.get("DH_FINAL"))
        te = r.get("TE")  # assume minutes
        try:
            te_min = float(te)
        except:
            te_min = 30.0
        jobs.append({
            "id": idx,
            "NUMOS": r.get("NUMOS"),
            "tipo": "tecnico",
            "coord": (float(r['LATITUDE']), float(r['LONGITUDE'])),
            "tw_start": tw_start,
            "tw_end": tw_end,
            "TE_min": te_min,
            "row": r
        })
        idx += 1
    # comerciais
    for _, r in df_comerciais.iterrows():
        tw_start = parse_dt(r.get("DATAINITRAB") or r.get("DATA_SOL"))
        tw_end   = parse_dt(r.get("DATATERTRAB") or r.get("DATA_VENC"))
        te = r.get("TE")
        try:
            te_min = float(te)
        except:
            te_min = 30.0
        jobs.append({
            "id": idx,
            "NUMOS": r.get("NUMOS"),
            "tipo": "comercial",
            "codserv": r.get("CODSERV"),
            "coord": (float(r['LATITUDE']), float(r['LONGITUDE'])),
            "tw_start": tw_start,
            "tw_end": tw_end,
            "TE_min": te_min,
            "row": r
        })
        idx += 1
    return jobs

def build_vehicles_from_eq(df_equipes: pd.DataFrame) -> List[Dict]:
    vehicles = []
    for i, r in df_equipes.iterrows():
        shift_start = parse_dt(r.get("dthaps_ini") or r.get("data_inicio_turno"))
        shift_end   = parse_dt(r.get("dthaps_fim_ajustado") or r.get("data_fim_turno") or r.get("dthaps_fim"))
        b_ini = parse_dt(r.get("dthpausa_ini"))
        b_fim = parse_dt(r.get("dthpausa_fim"))
        vehicles.append({
            "id": int(i),
            "equipe": r.get("equipe") or f"EQ{i}",
            "shift_start": shift_start,
            "shift_end": shift_end,
            "breaks": [(b_ini, b_fim)] if (b_ini and b_fim) else [],
            "start_coord": DEPOT,
            "end_coord": DEPOT,
            "row": r
        })
    return vehicles
# Observa: não trabalhar em horário de pausa; comerciais 739/741 entre 08-18; não atender job com DATA_SOL/DH_INICIO > shift_end (futuro)
def simulate_route_timeline(route_job_ids: List[int], jobs: List[Dict], vehicle: Dict, durations_matrix: np.ndarray, idx_map: Dict[int,int]) -> Tuple[bool, Dict]:
    """
    Simula execução sequencial da rota (job order given).
    idx_map: job_id -> position in durations_matrix (0 is depot)
    Returns (feasible, details) where details includes arrival, start, finish times for each job and totals.
    """
    cur_time = vehicle['shift_start']
    if cur_time is None or vehicle['shift_end'] is None:
        return False, {"reason":"no_shift_times"}
    prev = 0  # depot index in matrix
    schedule = []
    total_travel = 0.0
    total_service = 0.0
    for jid in route_job_ids:
        pos = idx_map[jid]
        travel_sec = durations_matrix[prev, pos]
        arrival = cur_time + timedelta(seconds=float(travel_sec))
        job = jobs[jid]
        # rule: no future job beyond shift_end
        if job['tw_start'] and job['tw_start'] > vehicle['shift_end']:
            return False, {"reason":"job_in_future", "job": job['NUMOS']}
        # if arrival before tw_start, wait
        if job['tw_start'] and arrival < job['tw_start']:
            arrival = job['tw_start']
        # comercial 739/741 only 8-18
        if job['tipo'] == 'comercial':
            cod = job.get('codserv')
            if cod in [739,741] or cod == '739' or cod == '741':
                if not (8 <= arrival.hour < 18):
                    return False, {"reason":"comercial_noite", "job":job['NUMOS']}
        # check break overlap
        for b in vehicle['breaks']:
            if b[0] and b[1]:
                start_service = arrival
                finish_service = arrival + timedelta(minutes=job['TE_min'])
                if (start_service < b[1]) and (finish_service > b[0]):
                    return False, {"reason":"overlaps_break", "job": job['NUMOS']}
        # check tw_end
        if job['tw_end'] and arrival > job['tw_end']:
            return False, {"reason":"tw_end_violation", "job": job['NUMOS']}
        finish = arrival + timedelta(minutes=job['TE_min'])
        # can return to depot before shift_end? we check after finishing last job; but to be safe ensure that after finishing the job and going back to depot we are <= shift_end
        ret_sec = durations_matrix[pos, 0]
        back_arrival = finish + timedelta(seconds=float(ret_sec))
        if back_arrival > vehicle['shift_end']:
            return False, {"reason":"cannot_return_before_shift_end", "job": job['NUMOS']}
        # append schedule item
        schedule.append({
            "job_id": jid,
            "numos": job['NUMOS'],
            "arrival": arrival,
            "start": arrival,
            "finish": finish
        })
        total_travel += float(travel_sec)
        total_service += job['TE_min']*60.0
        cur_time = finish
        prev = pos
    # final return
    total_travel += float(durations_matrix[prev,0])
    # ok
    details = {
        "schedule": schedule,
        "total_travel_sec": total_travel,
        "total_service_sec": total_service,
        "finish_time": schedule[-1]['finish'] if schedule else vehicle['shift_start']
    }
    return True, details
def compute_penalties_for_route(route_job_ids: List[int], jobs: List[Dict], details: Dict) -> float:
    """
    Returns penalty (unit: hours-equivalent multiplied into cost later).
    - técnico: (duração_da_interrupção * (EUSD/730) * 34)
      duração_da_interrupção: if DH_INICIO & DH_FINAL present -> use that duration_hours; else use TE_min/60
    - comercial: 120 + 34 * EUSD * log(prazo_verificado / prazo_regulatorio)
      prazo_verificado: (DATA_VENC - arrival).days (in days, min 1)
      prazo_regulatorio: use 7 days default (adjust if you have regulatory value)
    """
    penalty = 0.0
    for item in details.get("schedule", []):
        job = jobs[item['job_id']]
        if job['tipo']=='tecnico':
            row = job['row']
            dh_ini = parse_dt(row.get("DH_INICIO"))
            dh_fin = parse_dt(row.get("DH_FINAL"))
            if dh_ini and dh_fin:
                dur_hours = max(0.0, (dh_fin - dh_ini).total_seconds()/3600.0)
            else:
                dur_hours = job['TE_min']/60.0
            eusd = row.get("EUSD") or row.get("EUSD_FIO_B") or 1.0
            try:
                eusd_v = float(eusd)
            except:
                eusd_v = 1.0
            penalty += dur_hours * (eusd_v/730.0) * 34.0
        else:  # comercial
            row = job['row']
            eusd = row.get("EUSD") or row.get("EUSD_FIO_B") or 1.0
            try:
                eusd_v = float(eusd)
            except:
                eusd_v = 1.0
            venc = job.get("vencimento") or parse_dt(row.get("DATA_VENC"))
            arrival = item['arrival']
            if venc:
                prazo_verificado = max(1.0, (venc - arrival).total_seconds()/(3600.0*24.0))
                prazo_reg = 7.0
                try:
                    penalty += (235.97/2) + 15 * eusd_v * math.log(max(1.0, prazo_verificado/prazo_reg))
                except:
                    penalty += 120.0
            else:
                penalty += 120.0
    return penalty
# Representação: cromossomo = permutação dos job_ids com separators -1 per vehicle (end marker).
def random_chromosome(n_jobs, n_veh):
    """
    Retorna um cromossomo que representa uma atribuição de jobs para veículos.
    Se n_jobs < n_veh, algumas rotas serão vazias, o que é permitido.
    """
    # ordem aleatória dos jobs
    perm = list(range(n_jobs))
    random.shuffle(perm)

    # número de cortes real possível
    max_cuts = max(0, min(n_veh-1, n_jobs-1))

    if max_cuts > 0:
        cuts = sorted(random.sample(range(1, n_jobs), max_cuts))
    else:
        cuts = []

    chrom = []
    last = 0
    for c in cuts:
        chrom.append(perm[last:c])
        last = c
    chrom.append(perm[last:])

    # se faltarem equipes, preenche com rotas vazias
    while len(chrom) < n_veh:
        chrom.append([])

    return chrom

def decode_chromosome(chrom: List[int], n_veh: int) -> Dict[int, List[int]]:
    routes = {}
    vid = 0; cur = []
    for g in chrom:
        if g == -1:
            routes[vid] = cur.copy()
            cur = []
            vid += 1
            if vid>=n_veh:
                break
        else:
            cur.append(g)
    # pad if missing
    for v in range(n_veh):
        routes.setdefault(v, [])
    return routes

def fitness_of_chrom(chrom: List[int], jobs: List[Dict], vehicles: List[Dict], osrm_cache_coords: List[Tuple[float,float]]):
    """
    For each vehicle route (set of jobs; order not considered here), call VROOM to order them and then OSRM table to simulate timeline.
    We'll compute total cost = sum(total_travel_sec + penalty*3600).
    This is expensive (calls VROOM+OSRM), so: GA size must be moderate.
    """
    n_veh = len(vehicles)
    routes = decode_chromosome(chrom, n_veh)
    total_cost = 0.0
    # Pre-build mapping job_id -> coords
    for vid, job_list in routes.items():
        # Build VROOM payload for this single vehicle (vehicle info + jobs with time windows)
        if not job_list:
            continue
        veh = vehicles[vid]
        vroom_payload = {"vehicles": [], "jobs": []}
        vroom_vehicle = {
            "id": 1,
            "start": [veh['start_coord'][1], veh['start_coord'][0]],  # [lon,lat]
            "end": [veh['end_coord'][1], veh['end_coord'][0]],
            "time_window": [
                int(veh['shift_start'].timestamp()),
                int(veh['shift_end'].timestamp())
            ]
        }
        vroom_payload['vehicles'].append(vroom_vehicle)
        # jobs as per assigned set — VROOM will optimize order
        for j in job_list:
            job = jobs[j]
            tw = None
            if job['tw_start'] and job['tw_end']:
                tw = [int(job['tw_start'].timestamp()), int(job['tw_end'].timestamp())]
            vroom_job = {
                "id": int(job['id']),
                "location": [job['coord'][1], job['coord'][0]],
                "service": int(job['TE_min']*60),
            }
            if tw:
                vroom_job["time_window"] = tw
            vroom_payload['jobs'].append(vroom_job)
        # call VROOM to get route order
        try:
            vroom_resp = try_vroom_solve(vroom_payload, timeout=20)
        except Exception as e:
            # If VROOM fails, assign a high cost to this partition
            return 1e18
        # Parse VROOM response to get ordered list of job ids for this vehicle.
        # VROOM responses vary by version; try common patterns:
        ordered_job_ids = []
        # Pattern 1: vroom_resp has 'routes' list with 'steps' or 'jobs'
        if isinstance(vroom_resp, dict):
            # vroom standard sometimes returns 'routes' or 'solution' -> 'routes'
            if 'routes' in vroom_resp:
                # take first route
                route = vroom_resp['routes'][0]
                # route may contain 'steps' with 'job' or 'activities'
                # vroom also provides 'jobs' ordered in some responses
                if 'steps' in route:
                    for s in route['steps']:
                        if s.get('type')=='job':
                            ordered_job_ids.append(int(s.get('ref') or s.get('job')))
                elif 'jobs' in route:
                    ordered_job_ids = [int(x) for x in route['jobs']]
                elif 'activities' in route:
                    for act in route['activities']:
                        if act.get('type')=='job' and 'job' in act:
                            ordered_job_ids.append(int(act['job']))
            # fallback if vroom_resp returns 'solution' object
            if not ordered_job_ids and 'solution' in vroom_resp and 'routes' in vroom_resp['solution']:
                route = vroom_resp['solution']['routes'][0]
                if 'steps' in route:
                    for s in route['steps']:
                        if s.get('type')=='job':
                            ordered_job_ids.append(int(s.get('ref') or s.get('job')))
        # If still empty, attempt to parse common top-level keys
        if not ordered_job_ids:
            # try if response contains 'routes' as list of ints
            try:
                if 'route' in vroom_resp and isinstance(vroom_resp['route'], dict) and 'jobs' in vroom_resp['route']:
                    ordered_job_ids = [int(x) for x in vroom_resp['route']['jobs']]
            except:
                pass
        if not ordered_job_ids:
            # As final fallback, use the input job_list order
            ordered_job_ids = job_list.copy()

        # Now call OSRM table for coords in order: depot + ordered jobs
        coords_for_osrm = [DEPOT] + [jobs[j]['coord'] for j in ordered_job_ids]
        try:
            durations, distances = osrm_table(coords_for_osrm)
        except Exception as e:
            return 1e18

        # Build idx_map: job index -> position in durations (0 is depot, jobs start at 1)
        idx_map = {}
        for pos, j in enumerate(ordered_job_ids, start=1):
            # find job's global id index (it is j)
            idx_map[j] = pos

        # Simulate timeline based on OSRM durations and the VROOM order
        feasible, sim_details = simulate_route_timeline(ordered_job_ids, jobs, veh, durations, idx_map)
        if not feasible:
            # infeasible partition -> large cost
            return 1e17
        # compute penalty
        pen = compute_penalties_for_route(ordered_job_ids, jobs, sim_details)
        # cost: travel sec + penalty*3600 (scale penalty to seconds)
        cost_for_vehicle = sim_details['total_travel_sec'] + pen * 3600.0
        total_cost += cost_for_vehicle
    return total_cost

# Célula 10: GA main loop (moderado) + SA refinement on best individuals
def run_ga_assignment(jobs: List[Dict], vehicles: List[Dict], generations=40, pop_size=30):
    n_jobs = len(jobs); n_veh = len(vehicles)
    # precompute coords list to speed OSRM calls? we'll call OSRM per evaluation so caching would be complex; keep it simple
    pop = [random_chromosome(n_jobs, n_veh) for _ in range(pop_size)]
    fitness = [None]*pop_size
    for i,chrom in enumerate(pop):
        fitness[i] = fitness_of_chrom(chrom, jobs, vehicles, None)
    for gen in range(generations):
        # selection: tournament
        newpop = []
        for _ in range(pop_size//2):
            # select parents
            p1 = min(random.sample(pop, k=3), key=lambda c: fitness_of_chrom(c, jobs, vehicles, None))
            p2 = min(random.sample(pop, k=3), key=lambda c: fitness_of_chrom(c, jobs, vehicles, None))
            # crossover
            a,b = one_point_crossover_varlen(p1,p2)
            a = mutate_swap(a, 0.12)
            b = mutate_swap(b, 0.12)
            newpop.extend([a,b])
        # evaluate newpop
        pop = newpop
        fitness = [fitness_of_chrom(c, jobs, vehicles, None) for c in pop]
        # keep best printed
        best_idx = int(np.argmin(fitness))
        print(f"GA gen {gen} best {fitness[best_idx]:.2f}")
    best_chrom = pop[int(np.argmin(fitness))]
    best_routes = decode_chromosome(best_chrom, len(vehicles))
    return best_routes, best_chrom

# helpers: crossover / mutate
def one_point_crossover_varlen(a,b):
    if len(a)!=len(b):
        # pad shorter with random permutation to equalize length (rare)
        n = max(len(a), len(b))
        a2 = a[:] + random.sample([x for x in range(n) if x not in a], k=max(0,n-len(a)))
        b2 = b[:] + random.sample([x for x in range(n) if x not in b], k=max(0,n-len(b)))
        a,b = a2,b2
    pt = random.randint(1, len(a)-2)
    ca = a[:pt] + [x for x in b[pt:] if x not in a[:pt]]
    cb = b[:pt] + [x for x in a[pt:] if x not in b[:pt]]
    if ca[-1]!=-1: ca.append(-1)
    if cb[-1]!=-1: cb.append(-1)
    return ca, cb

def mutate_swap(chrom, rate=0.15):
    idxs = [i for i,g in enumerate(chrom) if g!=-1]
    for i in idxs:
        if random.random()<rate:
            j=random.choice(idxs); chrom[i],chrom[j]=chrom[j],chrom[i]
    return chrom

# Célula 11: função principal que roda dia a dia (P1), obrigando atribuição de todos os jobs.
def run_day(df_equipes: pd.DataFrame, df_tecnicos: pd.DataFrame, df_comerciais: pd.DataFrame, day: datetime.date, use_vroom=True):
    """
    - Filtra jobs whose date (DH_INICIO or DATA_SOL) falls on 'day'
    - Builds jobs and vehicles for that day
    - Runs GA to partition jobs among vehicles (assignment)
    - For each vehicle, calls VROOM to get order and OSRM table to compute times and simulate DH_FINAL
    - Returns structured summary including simulated DH_FINAL values
    """
    # filter by day
    def job_on_day(row, datecolnames):
        for c in datecolnames:
            v = parse_dt(row.get(c))
            if v and v.date() == day:
                return True
        return False

    df_t = df_tecnicos.copy()
    df_c = df_comerciais.copy()
    df_t = df_t[df_t.apply(lambda r: job_on_day(r, ["DH_INICIO","DH_ALOCACAO","DH_CHEGADA"]), axis=1)].reset_index(drop=True)
    df_c = df_c[df_c.apply(lambda r: job_on_day(r, ["DATA_SOL","DATAINITRAB"]), axis=1)].reset_index(drop=True)

    jobs = build_jobs_from_dfs(df_t, df_c)
    vehicles = build_vehicles_from_eq(df_equipes)

    if not jobs:
        print("Nenhum job para este dia:", day)
        return {}

    # Run GA assignment (note: expensive; adjust generations/pop_size as needed)
    print(f"Rodando GA de atribuição para {len(jobs)} jobs e {len(vehicles)} veículos...")
    best_routes, best_chrom = run_ga_assignment(jobs, vehicles, generations=20, pop_size=20)

    # For each vehicle, get VROOM order and then OSRM times to simulate final times
    summary = {"day": str(day), "vehicles": []}
    for vid, job_list in best_routes.items():
        veh = vehicles[vid]
        if not job_list:
            summary["vehicles"].append({"vehicle_id": vid, "equipe": veh['equipe'], "route": [], "details": {}})
            continue
        # build VROOM payload (vehicle + jobs)
        vroom_payload = {"vehicles": [], "jobs": []}
        vroom_vehicle = {
            "id": 1,
            "start": [veh['start_coord'][1], veh['start_coord'][0]],
            "end": [veh['end_coord'][1], veh['end_coord'][0]],
            "time_window": [
                int(veh['shift_start'].timestamp()),
                int(veh['shift_end'].timestamp())
            ]
        }
        vroom_payload['vehicles'].append(vroom_vehicle)
        for j in job_list:
            job = jobs[j]
            job_payload = {
                "id": int(job['id']),
                "location": [job['coord'][1], job['coord'][0]],
                "service": int(job['TE_min']*60)
            }
            if job['tw_start'] and job['tw_end']:
                job_payload["time_window"] = [int(job['tw_start'].timestamp()), int(job['tw_end'].timestamp())]
            vroom_payload['jobs'].append(job_payload)

        # call VROOM to get order
        if use_vroom:
            try:
                vroom_resp = try_vroom_solve(vroom_payload, timeout=20)
            except Exception as e:
                print("VROOM call failed:", e)
                # fallback to original ordering
                ordered_job_ids = job_list.copy()
            else:
                # parse vroom response to ordered ids (similar parsing as before)
                ordered_job_ids = []
                if 'routes' in vroom_resp:
                    route = vroom_resp['routes'][0]
                    if 'steps' in route:
                        for s in route['steps']:
                            if s.get('type')=='job':
                                ref = s.get('ref') or s.get('job') or s.get('id')
                                if ref is not None:
                                    ordered_job_ids.append(int(ref))
                    elif 'jobs' in route:
                        ordered_job_ids = [int(x) for x in route['jobs']]
                    elif 'activities' in route:
                        for act in route['activities']:
                            if act.get('type')=='job' and 'job' in act:
                                ordered_job_ids.append(int(act['job']))
                if not ordered_job_ids:
                    ordered_job_ids = job_list.copy()
        else:
            ordered_job_ids = job_list.copy()

        # OSRM table for depot + ordered jobs
        coords_list = [DEPOT] + [jobs[j]['coord'] for j in ordered_job_ids]
        durations, distances = osrm_table(coords_list)  # may raise if OSRM not reachable
        # build index map
        idx_map = {j: pos for pos,j in enumerate(ordered_job_ids, start=1)}

        feasible, details = simulate_route_timeline(ordered_job_ids, jobs, veh, durations, idx_map)
        if not feasible:
            summary["vehicles"].append({"vehicle_id": vid, "equipe": veh['equipe'], "route": ordered_job_ids, "details": {"feasible": False, "reason": details}})
            continue
        # compute penalties
        pen = compute_penalties_for_route(ordered_job_ids, jobs, details)
        # compute new DH_FINAL for each job as requested:
        # "final será dthaps_ini + tempo de chegada estimado + (TE em minutos)"
        # We'll compute an estimated DH_FINAL_sim for each job: vehicle.shift_start + travel_to_job + TE_min
        est_new_final = []
        # compute cumulative travel times along ordered list using durations matrix
        cur_pos = 0
        cur_time = veh['shift_start']
        for jpos in ordered_job_ids:
            pos = idx_map[jpos]
            travel_sec = durations[cur_pos, pos]
            arrival = cur_time + timedelta(seconds=float(travel_sec))
            # DH_FINAL_sim = dthaps_ini + tempo_chegada_estimado + TE_min
            DH_FINAL_sim = veh['shift_start'] + (arrival - veh['shift_start']) + timedelta(minutes=jobs[jpos]['TE_min'])
            est_new_final.append({"job_id": jpos, "NUMOS": jobs[jpos]['NUMOS'], "DH_FINAL_sim": DH_FINAL_sim})
            # update cur_time = finish
            cur_time = arrival + timedelta(minutes=jobs[jpos]['TE_min'])
            cur_pos = pos

        vehicle_summary = {
            "vehicle_id": vid,
            "equipe": veh['equipe'],
            "ordered_job_ids": ordered_job_ids,
            "osrm_total_travel_sec": details['total_travel_sec'],
            "osrm_total_service_sec": details['total_service_sec'],
            "penalty": pen,
            "sim_details": details,
            "DH_FINAL_sim_list": est_new_final
        }
        summary["vehicles"].append(vehicle_summary)
    # Mark all jobs assigned check
    assigned = []
    for v in summary['vehicles']:
        assigned.extend(v.get("ordered_job_ids", []))
    assigned_set = set(assigned)
    all_jobs_set = set([j['id'] for j in jobs])
    if assigned_set != all_jobs_set:
        missing = all_jobs_set - assigned_set
        print("Atenção: nem todos os jobs foram atribuídos! faltam:", missing)
    else:
        print("Todos os jobs atribuídos com sucesso.")
    return summary

# Célula 12: exemplo de execução
# Substitua abaixo pelos seus DataFrames carregados
# Exemplo: df_equipes = pd.read_csv("equipes.csv", parse_dates=[...]); df_tecnicos = ...; df_comerciais = ...

# Para demonstração, você deve carregar aqui seus dataframes reais. Exemplo:
df_equipes = pd.read_parquet("data/Equipes.parquet")
df_tecnicos = pd.read_parquet("data/atendTec.parquet")
df_comerciais = pd.read_parquet("data/ServCom.parquet")

# Depois, rode por dia (exemplo: dias encontrados nas colunas DH_INICIO / DATA_SOL)
# Aqui apenas um esqueleto para chamar run_day:

day = datetime(2023, 5, 1).date()
summary_day = run_day(df_equipes, df_tecnicos, df_comerciais, day, use_vroom=True)
from pprint import pprint
pprint(summary_day)


### utilitários de datas e distâncias

### funções para tentar o endpoint VROOM (robustez)

### função para pedir matriz OSRM (table) para um conjunto de coords

### Build jobs/vehicles from seus DataFrames (assumindo nomes das colunas que você passou)

### regra de viabilidade para uma rota (dado ordem de atendimento).

### penalizações (conforme Ren1000 e Prodist)

### GA para criar uma partição (atribuição) dos jobs entre veículos.

### GA main loop (moderado) + SA refinement on best individuals

### função principal que roda dia a dia (P1), obrigando atribuição de todos os jobs

### execução